In [0]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [3]:
from google.colab import drive
drive.mount("/content/drive/")

project_path = "/content/drive/My Drive/StaticalNLP/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [4]:
yelp = pd.read_csv(project_path+"Day1_snlp/yelp.csv")
yelp.head(5)

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [0]:
yelp["sentiment"] = yelp["stars"]

In [0]:
yelp["sentiment"] = yelp["sentiment"].replace([1,2,3,4,5], ["bad","bad","neutral","good","good"])

In [7]:
yelp.stars.value_counts()

4    3526
5    3337
3    1461
2     927
1     749
Name: stars, dtype: int64

In [8]:
yelp.sentiment.value_counts()

good       6863
bad        1676
neutral    1461
Name: sentiment, dtype: int64

In [0]:
X = yelp[["text", "sentiment"]]

In [0]:
vect = CountVectorizer(ngram_range=(1,2), stop_words='english', min_df=0.01)

In [0]:
vect.fit(X["text"])
X_features = vect.transform(X["text"])

In [13]:
X_features.shape

(10000, 1010)

In [0]:
features = np.array(vect.get_feature_names())

In [0]:
feature_len = np.array([len(x.split(" ")) for x in features])

In [16]:
features[np.where(feature_len == 2)]

array(['chips salsa', 'customer service', 'don know', 'don like',
       'don think', 'fast food', 'feel like', 'felt like', 'food good',
       'food great', 'food service', 'friday night', 'friendly staff',
       'good food', 'good place', 'good service', 'good time',
       'great food', 'great place', 'great service', 'happy hour',
       'highly recommend', 'ice cream', 'just like', 'just right',
       'like place', 'little bit', 'long time', 'looks like',
       'love place', 'mac cheese', 'make sure', 'mexican food',
       'old town', 'parking lot', 'place good', 'place great',
       'pretty good', 'really good', 'really like', 'really nice',
       'reasonably priced', 'service good', 'service great',
       'staff friendly', 'strip mall', 'sweet potato', 'tasted like',
       'wait staff', 'years ago'], dtype='<U17')

In [17]:
type(X_features)

scipy.sparse.csr.csr_matrix

In [0]:
dtm = pd.DataFrame(X_features.toarray())

In [0]:
dtm.columns = vect.get_feature_names()

In [20]:
dtm[dtm["customer service"] != 0].index

Int64Index([  13,   19,   55,   61,  113,  148,  155,  184,  348,  355,
            ...
            9538, 9557, 9597, 9636, 9767, 9773, 9790, 9801, 9872, 9880],
           dtype='int64', length=228)

In [21]:
X.text.iloc[13]

"Good tattoo shop. Clean space, multiple artists to choose from and books of their work are available for you to look though and decide who's style most mirrors what you're looking for. I chose Jet to do a cover-up for me and he worked with me on the design and our ideas and communication flowed very well. He's a very personable guy, is friendly and keeps the conversation going while he's working on you, and he doesn't dick around (read: He starts to work and continues until the job is done). He's very professional and informative. Good customer service combines with talent at the craft."

Punctuation handling while tokenization, removing stopwords etc

In [0]:
text = ["applicant should be proficient in Go","need experience in node.js","expert in C & C#", "working on go to market strategy"]

In [0]:
vect = CountVectorizer(stop_words='english')


In [0]:
dtm = pd.DataFrame(vect.fit_transform(text).toarray())
dtm.columns = vect.get_feature_names()

In [25]:
dtm

,applicant,experience,expert,js,market,need,node,proficient,strategy,working
0,1,0,0,0,0,0,0,1,0,0
1,0,1,0,1,0,1,1,0,0,0
2,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,1,1


In [0]:
#from sklearn.feature_extraction import stop_words
#print(stop_words.ENGLISH_STOP_WORDS)

In [0]:
import re

In [27]:
text_mode = [re.sub("C#", "CSharp", x, flags=re.IGNORECASE) for x in text]
text_mode = [re.sub("Go", "GoLang", x) for x in text_mode]
text_mode

['applicant should be proficient in GoLang',
 'need experience in node.js',
 'expert in C & CSharp',
 'working on go to market strategy']

In [0]:
vect = CountVectorizer(tokenizer=lambda x: x.split(" "))
dtm = pd.DataFrame(vect.fit_transform(text_mode).toarray())
dtm.columns = vect.get_feature_names()

In [29]:
dtm

,&,applicant,be,c,csharp,experience,expert,go,golang,in,market,need,node.js,on,proficient,should,strategy,to,working
0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0
1,0,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0
2,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,1


Reducing inflectional forms via Stemming & Lemmatization

In [0]:
text = ["loved the movie","love the acting and the cast","i am loving it"]

In [0]:
dtm = pd.DataFrame(vect.fit_transform(text).toarray())
dtm.columns = vect.get_feature_names()

In [32]:
dtm

,acting,am,and,cast,i,it,love,loved,loving,movie,the
0,0,0,0,0,0,0,0,1,0,1,1
1,1,0,1,1,0,0,1,0,0,0,2
2,0,1,0,0,1,1,0,0,1,0,0


In [33]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
import nltk

from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [0]:
stem = PorterStemmer()

In [36]:
stem.stem("loving")

'love'

In [0]:
text_stem = []
for review in text:
    text_stem.append(" ".join([stem.stem(x) for x in word_tokenize(review)]))

In [38]:
text_stem

['love the movi', 'love the act and the cast', 'i am love it']

Lemmatization

In [40]:
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')
pos_tag(word_tokenize("he works in banking"))
pos_tag(word_tokenize("i am banking on you to do this"))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


[('i', 'NN'),
 ('am', 'VBP'),
 ('banking', 'VBG'),
 ('on', 'IN'),
 ('you', 'PRP'),
 ('to', 'TO'),
 ('do', 'VB'),
 ('this', 'DT')]

In [0]:
lemma = WordNetLemmatizer()

In [52]:

lemma.lemmatize("banking", "v")

'bank'

In [53]:
from nltk.corpus import wordnet
lemma.lemmatize("better",wordnet.ADJ)

'good'

In [48]:
import nltk
nltk.download("popular")


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [0]:
def get_wordnet_pos_tag(tag):
    if tag.startswith("J"):
        return wordnet.ADJ
    if tag.startswith("V"):
        return wordnet.VERB
    if tag.startswith("N"):
        return wordnet.NOUN
    if tag.startswith("R"):
        return wordnet.ADVERB
    else:
        return wordnet.NOUN

In [55]:
lemma.lemmatize("loved",get_wordnet_pos_tag("VBD"))

'love'

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test, X_train_orig, X_test_orig = train_test_split(X_features, X["sentiment"], X["text"],
                                                                             random_state=2)


In [0]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [58]:
nb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
y_pred_class = nb.predict(X_test)

In [0]:
pred_df = pd.DataFrame({"actual":y_test, "pred":y_pred_class})

In [61]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.7532

In [62]:
pd.crosstab(pred_df["actual"],pred_df["pred"])

pred,bad,good,neutral
actual,,,
bad,264,98,68
good,97,1483,141
neutral,54,159,136
